In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
import os
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent))
print(f"Folder \033[1m{sys.path[-1]}\033[0m added to PATH")

In [ ]:
INPUT_FILE = '../sample_data/UC_Berkeley.pdf'
QA_DATASET = '../sample_data/rag/'
FINETUNED_MODEL = 'llama-2-7b-berkeley'

## Generate dataset

In [ ]:
from generate_dataset import main as generate_dataset_main
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_openai.chat_models import ChatOpenAI

In [ ]:
llm_critic = ChatOpenAI(model="gpt-4-0125-preview")
llm_emb = OpenAIEmbeddings(model="text-embedding-3-small")

generate_dataset_main(llm_critic, llm_emb, file_path=INPUT_FILE, output=QA_DATASET,
                      chunk_size=512, num_questions=3, max_chunks=2)

## RAG Model Fine tuning 

In [ ]:
from finetune_model import main as finetune_main, DEFAULT_MODEL_NAME

In [ ]:
finetune_main(QA_DATASET, DEFAULT_MODEL_NAME, FINETUNED_MODEL)

## Model evaluation

In [ ]:
from finetune_model import get_tokenizer_model, format_instruction
from datasets import load_from_disk
from transformers import pipeline

model, tokenizer = get_tokenizer_model(FINETUNED_MODEL, inference_mode=True)
dataset = load_from_disk(QA_DATASET)

In [ ]:
# Generate response
test = dataset[1]
del(test["prompt_cot_response"])
query = format_instruction(test)
print(query)
text_gen = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1024, temperature=0)
output = text_gen(query)
print(f"\033[92m{output[0]['generated_text']}\033[0m")